### Selection of terms in the dictionary
What are words? Perhaps too heavy an epistomological question before coffee or alcohol... Thankfully there is a vsauce video to help discuss the [Zipf function](https://www.youtube.com/watch?v=fCn8zs912OE) and some of its implications.

Some works offer more "useful" information than others for many tasks. If the subject of a conversation is inherant, the term "I" is redundant. It is common in many languages for its equivalent to be absent. This is the first of many terms that offer limited information for determining what the document concerns. Generally in knowledge extraction and classification tasks we remove these [stop-words](https://en.wikipedia.org/wiki/Stop_words) as they clutter the vector representations. The common practice is to remove these stop-words as by doing so, we most often improve the accuracy of the model that follows.

On the opposite end of Zipf's distribution are the terms that are used very infrequently. Does the inclusion of the term "parsimonius" in your dictionary help you improve a model? Perhaps if the object of your model is to separate documents written by academics and everyone else it may be useful... However, if it occurs infrequently within a corpus it represents an outlier in the data. Any model we develop against a corpus will include these infrequent terms. In doing so the model will fit the training data more closely, but therein lies the problem. It will fit the training data and not necessarily the real data. 

Determining where to cull most frequent and infrequent terms is ultimately a question of the language, the dataset, and the model being used. Build the model based on a best estimate, revise the dictionary, rebuild the model, repeat, and graph the outcomes. If the model fit quickly becomes poor with further reduction, stop.

### Term frequency inverse document freqency
Where we consider each document as a vector we are treating the occurance of each word as having equal value. Clearly, this is not the consistent with our experience of language. "He was angry" carries a different weight than "He was apoplectic" and yet they carry the same basic meaning. Knowing this, we can transform or re-weight these vectors after corpus construction. The value of doing so is dependant on the modeling that we subsequently perform. For example for neural networks, the network itself is being trained to effectively weight each term.

Term frequency inverse document frequency weighting, or [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) as it is more commonly known, is a method for interpreting the value of a word based on its occurrance in the corpus and the document. Term frequency is simply:

$tf(t, d) = f_{t, f}$

Which is simply the count of term $t$ in document $d$. The rationale being that "really really mad" is more significant than being "really mad". The inverse document requires the consideration of how commonly a term is used within the corpus as a whole. Logarithmically scaled, the inverse document frequency is as follows:

$idf(t, D) = \log \frac{N}{\{d \in D: t \in d\}}$

So for a given term $t$ in the corpus of documents $D$, we take the $\log$ of the total number of documents $N$ divided by the number of documents where that term appears. To avoid division by zero, we can add unity, but we would normally avoid this in the construction of the dictionary.

In [1]:
# import useful stuff
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
with open('carmilla.txt', 'r') as f:
    corpus = f.read()

### Exercise 02
Given Carmilla: 
- Construct a dictionary of all terms/words present. Convert characters to lowercase as you do so.
- Plot the distribution of the terms.
- Construct a list of the stop-words and remove them from the dictionary.
- Remove the least frequent terms from the dictionary.
- From the documents at the paragraph level, calculate the tf-idf of each term.
- Transform the bag-of-words vector representation of the documents into the tf-idf weightings.

Note that we will only use the tuple format for bag-of-words for larger corpora. It may be useful to build a transformation function so that when we start working with scikit-learn the transition is simple.

In [3]:
from utils.cheaters import dctConstr

dct = dctConstr(stop_words=["i", "you", "a"], ignore_case=True)
dct.constructor(corpus)
# dct.trimmer(top=5, bottom=10)
# dct.build_tfidf(corpus)

def split_by_paragraphs(data:str) -> []:
    processed=data.lower()
    while '\n\n\n' in processed:
        processed=processed.replace('\n\n\n','\n\n')
    out = processed.split('\n\n')
    return [o.replace("\n", " ") for o in out]

pcorp = split_by_paragraphs(corpus)
print(len(pcorp))
pvect = [dct.tfidf(para) for para in pcorp]

676


In [4]:
print(pcorp[30])
print(pvect[30])
dct("\n _which")

but i was not comforted, for i knew the visit of the strange woman was _not_ a dream; and i was _awfully_ frightened.
[(9, 0.004166666666666667), (13, 0.008333333333333333), (31, 0.004166666666666667), (42, 0.004166666666666667), (50, 0.004166666666666667), (53, 0.004166666666666667), (61, 0.0125), (92, 0.008333333333333333), (102, 0.0125), (425, 0.004166666666666667), (492, 0.004166666666666667), (660, 0.004166666666666667), (662, 0.004166666666666667), (663, 0.004166666666666667), (664, 0.004166666666666667), (665, 0.004166666666666667), (666, 0.004166666666666667), (3987, 0.004166666666666667)]


[(15, 1), (3987, 1)]